In [ ]:
"""
현재까지 만들어진 이미지 객체들
hero : 본체
bullet : 총알
enemy : 적군
===============================
bulList : 총알 저장 리스트
enemList: 적군 저장 리스트
"""


import pygame
import random
from datetime import datetime      #경과시간을 위해
import time as t

"""
1. 게임 초기화
"""

pygame.init()

"""
2. 게임 창 옵션 설정
"""


#크기 설정
size = [800,1000]                         
screen = pygame.display.set_mode(size)

 #제목 설정
title = "Save Earth"                    
pygame.display.set_caption(title)        



"""
3. 게임 내 필요한 설정
"""

#시계 만들기
clock = pygame.time.Clock()              

"""
이미지 파일
"""
backgroundImg = pygame.image.load(r'image\background1.jpg')
backgroundImg = pygame.transform.scale(backgroundImg, (800, 1000))

logoImg = pygame.image.load(r'image\logo.png').convert_alpha()
logoImg = pygame.transform.scale(logoImg, (600, 500))

border = pygame.image.load(r'image\border.png').convert_alpha()
border = pygame.transform.scale(border, (800, 100))


"""
오디오 파일
"""
hit_sound = pygame.mixer.Sound(r'sound\hit.wav')
break_sound = pygame.mixer.Sound(r'sound\break.wav')
item_sound = pygame.mixer.Sound(r'sound\item.wav')
start_sound = pygame.mixer.Sound(r'sound\start.wav')
end_sound = pygame.mixer.Sound(r'sound\end.wav')
standby_sound = pygame.mixer.Sound(r'sound\standby.wav')
game_background_sound = pygame.mixer.Sound(r'sound\game_background.wav')
heartBeet_sound = pygame.mixer.Sound(r'sound\heartBeet.wav')

end_sound.set_volume(0.5)
start_sound.set_volume(0.4)
standby_sound.set_volume(0.1)
game_background_sound.set_volume(0.1)
item_sound.set_volume(1)
heartBeet_sound.set_volume(1)
hit_sound.set_volume(0.1)
break_sound.set_volume(0.1)

#클래스 생성
class createObj:
    #생성자
    def __init__(self):
        self.x = 0
        self.y = 0
        self.move = 0
        self.kill = 0
        self.delay = 0
        self.tic=0
        self.power=0
    #캐릭터 입력
    def insertImg(self, addr):
        if addr[-3:] == 'png':
            self.obj = pygame.image.load(addr).convert_alpha()
        else:
            self.obj = pygame.image.load(addr)
        #캐릭터 x길이, y길이 받아오기
        self.oX, self.oY = self.obj.get_size()
        
    #사진 사이즈 설정
    def setSize(self, oX, oY):
        self.obj = pygame.transform.scale(self.obj, (oX, oY))
        #캐릭터 x길이, y길이 받아오기
        self.oX, self.oY = self.obj.get_size()
    
    #blit
    def show(self):
        screen.blit(self.obj, (self.x, self.y))

        
        
#충돌 탐지 함수
"""
충돌 탐지 함수 crashDetec ()
서로 다른 이미지들이 충돌하면 True값을 반환하는 함수입니다.

ex) a=crashDetec(hero,enemy)
>>>print(a)
>>>True
"""
def crashDetec(vic,perp):
    if vic.x-perp.oX+5 < perp.x and perp.x < vic.x+vic.oX-5:
        if vic.y-perp.oY+5 < perp.y and perp.y < vic.y+vic.oY-5:
            return True
        else:
            return False
    else:
        return False

    
"""
충돌 좌표 함수 crashCoor()
서로 다른 이미지들이 충돌하면 충돌 좌표를 반환하는 함수입니다.
"""
def crashCoor(vic,perp):
    if vic.x-perp.oX+5 < perp.x and perp.x < vic.x+vic.oX-5:
        if vic.y-perp.oY+5 < perp.y and perp.y < vic.y+vic.oY-5:
            return perp.x, perp.y

"""
게임 종료 함수 end_game(point,shootDown,missed,sub_time,game_over,clear)
"""
def end_game(point,shootDown,missed,sub_time,game_over,clear=False):
    heartBeet_sound.stop()
    game_background_sound.stop()
    end_sound.play()
    """
    0. 게임 종료 
    """
    point += sub_time*5

    while game_over == 1:
        #FPS 설정 초당 반응 수
        clock.tick(60)

        """
        0-1. 입력 감지
        """
        #스페이스가 눌리면 게임 시작
        for event in pygame.event.get():
            #x버튼 누르면 종료
            if event.type == pygame.QUIT:
                game_over=0

        """
        0-2. 그리기
        """
        #텍스트 폰트 설정
        font = pygame.font.Font(r"font\경기천년제목V_Bold.ttf",70)
        font2 = pygame.font.Font(r"font\경기천년제목V_Bold.ttf",30)
        font3 = pygame.font.Font(r"font\경기천년제목V_Bold.ttf",40)

        """
        글자 생성
        """
        if clear == True:     #게임 클리어 시
            end_textG = font.render("GAME CLEAR",True, (255,0,0))
        else:
            end_textG = font.render("GAME OVER",True, (255,0,0))

        end_textS = font2.render(f"Shoot Down",True, (0,128,0))
        end_textSP = font2.render(f"{shootDown}  x  100",True, (0,0,255))
        end_textM = font2.render(f"Missed",True, (0,128,0))
        end_textMP = font2.render(f"{missed}  x  -50",True, (0,0,255))
        end_textT = font2.render(f"Play Time",True, (0,128,0))
        end_textTP = font2.render(f"{sub_time}  x  5",True, (0,0,255))

        end_textP = font3.render(f"Point",True, (0,128,0))
        end_textPP = font3.render(f"{point}p",True, (0,0,255))

        bar = font.render("|",True, (255,255,255))
        topBar = font.render("===========",True, (255,255,255))
        bottomBar = font.render("------------",True, (255,255,255))

        bottomBar2 = font2.render("------------------------",True, (255,255,255))
        #텍스트 표시
        screen.blit(topBar, (207,250))
        screen.blit(end_textG, (207,300))
        screen.blit(end_textS, (220,400))
        screen.blit(end_textSP, (450,400))
        screen.blit(end_textM, (220,450))
        screen.blit(end_textMP, (450,450))
        screen.blit(end_textT, (220,500))
        screen.blit(end_textTP, (450,500))
        screen.blit(bottomBar2, (220,540))
        screen.blit(end_textP, (220,600))
        screen.blit(end_textPP, (420,600))

        screen.blit(bar,(180,270))
        screen.blit(bar,(620,270)) 
        screen.blit(bar,(180,320))
        screen.blit(bar,(620,320))
        screen.blit(bar,(180,370))
        screen.blit(bar,(620,370))
        screen.blit(bar,(180,420))
        screen.blit(bar,(620,420))
        screen.blit(bar,(180,470))
        screen.blit(bar,(620,470))
        screen.blit(bar,(180,520))
        screen.blit(bar,(620,520))
        screen.blit(bar,(180,570))
        screen.blit(bar,(620,570))
        screen.blit(bar,(180,620))
        screen.blit(bar,(620,620))
        screen.blit(bottomBar,(199,650))
        
        """
        0-3. 업데이트
        """
        #screen 상태 업데이트
        pygame.display.flip()
    

    
"""
=======================
hero 객체 생성 및 설정
"""
hero = createObj()
#캐릭터 경로 설정
hero.insertImg(r'image\hero.png')
#캐릭터 크기 설정
hero.setSize(60,90)
#캐릭터 위치 설정
hero.x = round(size[0]/2 - hero.oX/2)
hero.y = size[1] - 50 - hero.oY
hero.move = 9
hero.power = 1
"""
================
"""


"""
대기화면 0. 시작
"""

"""
대기화면 1. 변수 설정
"""
#버튼 누르면 startB값이 0이되어 종료
startB = 1

#텍스트 색
text_index = 0


#startB가 1이 아니라면 while문 종료
while startB == 1:
    #FPS 설정 초당 반응 수
    clock.tick(60)
    # 대기화면 배경음악 시작 및 반복
    standby_sound.play(-1)
    """
    대기화면 2. 입력 감지
    """
    #스페이스가 눌리면 게임 시작
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == 32 or event.key == 107:            #press on space bar
                standby_sound.stop()
                start_sound.play()
                
                startB = 0
    
    """
    대기화면 3. 값의 변화
    """
    text_index+=1
    if text_index == 36000:
        text_index=0
    
    

            
    
    """
    대기화면 4. 그리기
    """
    #배경사진 넣기
    screen.blit(backgroundImg,(0,0))
    #로고사진 넣기
    screen.blit(logoImg,(100,70))
    
    #텍스트 폰트 설정
    font = pygame.font.Font(r"font\경기천년제목V_Bold.ttf",20)
    #스페이스 누르라는 텍스트
    if text_index // 9 % 2 == 0:
        start_text = font.render("Press SPACE BAR to start game!",True, (255,255,255))
    else:
        start_text = font.render("Press SPACE BAR to start game!",True, (128,128,128))
    #텍스트 표시
    screen.blit(start_text, (250,round(size[1]/2)+300))
    
    """
    대기화면 5. 업데이트
    """
    
    #screen 상태 업데이트
    pygame.display.flip()  




"""
Game 0. 시작
"""

"""
Game 1. 변수 설정
"""
game_background_sound.play(-1)
#시작 현재 시간 불러오기
start_time = datetime.now()

"""
1.1 종료 관련 변수
"""
end = 1               #버튼 누르면 end값이 0이되어 종료
game_over = 0         #game_over가 1이 되면 게임오버 창이 나오도록


"""
1.2 이동 관련 변수
"""
###초기화 설정###
#이동 값(bool) 초기화
leftMove=False
rightMove=False
topMove = False
bottomMove = False


"""
1.3 포인트에 관한 변수
"""
shootDown=0  #적기 격추 = shootDown
missed=0     #적기 놓침 = missed
point=0      #포인트값 point = (shootDown*100)-(missed-50)


"""
1.4 hero에 관한 변수
"""
lifeList=[]         #생명 리스트
invin_tic=0         #무적 시간 체크
invinTime = 3       #무적 시간
invin = False       #무적 상태 유무

"""
1.5 캐릭터 체력 화면 표시 정의 구간
"""
lifeNum=5             #캐릭터 HP
for i in range(lifeNum):
    life = createObj()
    life.insertImg(r'image\life.png')
    life.setSize(40,40)
    life.x = 105 + i*50
    life.y = 5
    lifeList.append(life)


"""
1.6 총알에 관한 변수
"""
bulList=[]          #총알 리스트
k=0                 #k : 총알 속도 연관 변수1
bullet_speed = 10    # 총알 속도 관련 변수2 작아질 수록 빨라짐
shot = False        #슈팅 여부


"""
1.7 적군에 관한 변수
"""
enemList=[]               # 적군 리스트
enemy_speed = [1,2,3,4]   # 적군 랜덤 속도 리스트, 시간이 지날 수록 빨라짐
deathNum=1                # 적군 HP
change_time = 35          # 적군 HP가 변하는 시간
enem_num_index = 0.97     # 적군 생성 빈도 값
enem_num_check = 1        # 증가 인덱싱을 통해서 조절
shootDown_check = -1
update_deathNum = False   #deathNum 업데이트 유무


"""
1.8 효과 관련 변수
"""
boomList=[]    #처치 효과 리스트
redList=[]     #피격 효과 리스트
hitList=[]     #타격효과 리스트
hit_idx=0      #타격 효과 화면 표시 인덱스


"""
1.9 아이템에 관한 변수
"""
healPlusList = []      #체력 추가 리스트
powerUpList = []       #파워업 추가 리스트
speedUpList = []       #스피드업 추가 리스트
makeItem = 17           #아이템 생성 격추 인덱스
itemCheck=False        #아이템 생성 됐는가 여부


#end가 1이 아니라면 while문 종료
while end == 1:
    """
    Game 2. FPS 설정
    """
    #FPS 설정 초당 반응 수
    clock.tick(60)                       

    """
    Game 3. 각종 입력 감지
    """
    
    #입력 감지
    
    for event in pygame.event.get():
        #x버튼 누르면 종료
        if event.type == pygame.QUIT:
            end=0
            
        """
        ============================================================================================================
        3.1 wasd or 방향키 입력 or 슈팅 입력(space bar or k) 감지
        """
        #KeyDown되면 활성화
        if event.type == pygame.KEYDOWN:
            if event.key == 97 or event.key == 1073741904:     #press on a
                leftMove = True
            if event.key == 100 or event.key == 1073741903:    #press on d
                rightMove = True
            if event.key == 115 or event.key == 1073741905:    #press on s
                bottomMove = True
            if event.key == 119 or event.key == 1073741906:    #press on w
                topMove = True
            if event.key == 32 or event.key == 107:            #press on space bar
                shot = True
                k=0
        #KeyUp 되면 비활성화
        elif event.type == pygame.KEYUP:
            if event.key == 97 or event.key == 1073741904:     #press off a
                leftMove = False
            if event.key == 100 or event.key == 1073741903:    #press on d
                rightMove = False
            if event.key == 115 or event.key == 1073741905:    #press off s
                bottomMove = False
            if event.key == 119 or event.key == 1073741906:    #press off w
                 topMove = False
            if event.key == 32 or event.key == 107:            #press off space bar
                shot = False 
        """
        ============================================================================================================
        """
    
    
    """
    Game 4. 입력과 시간에 따른 변화
    """
    #게임 진행 시 현재 시간 불러오기
    now_time = datetime.now()
    #진행되는 시간 구하기
    sub_time = round((now_time - start_time).total_seconds(),1)
    """
    ============================================================================================================
    4.1 캐릭터 이동에 관한 부분 정의
    """
    #좌측 이동
    if leftMove == True:
        hero.x -= hero.move
        #화면에 벗어나지 않도록
        if hero.x <= 0:
            hero.x = 0
    #우측 이동
    if rightMove == True:
        hero.x += hero.move
        #화면에 벗어나지 않도록
        if hero.x >=size[0]-hero.oX:
            hero.x = size[0]-hero.oX
    #위로 이동
    if topMove == True:
        hero.y -= hero.move
        #화면에 벗어나지 않도록
        if hero.y <= 100 :
            hero.y = 100
    #아래로 이동
    if bottomMove == True:
        hero.y += hero.move
        #화면에 벗어나지 않도록
        if hero.y >= size[1]-hero.oY:
            hero.y = size[1]-hero.oY
    """
    ============================================================================================================
    """

    """
    ============================================================================================================
    4.2 캐릭터가 총알 쏠 때 
    bul_num의 변수 값의 k % n 중 n값이 작아질 수록 총알 빨라짐.
    """
    bul_num = k % bullet_speed 
    if shot == True and bul_num == 0:
        hit_sound.stop()
        hit_sound.play()
        bullet = createObj()
        bullet.insertImg(r'image\bullet.png')
        bullet.setSize(30,50)
        bullet.x = round(hero.x + hero.oX/2 - bullet.oX/2)
        bullet.y = hero.y - bullet.oY + 20
        bullet.move = 20
        bulList.append(bullet)
    k+=1
    
    # 생성된 총알 움직이기
    del_bulList=[]
    for i,v in enumerate(bulList):
        v.y -= v.move
        # 총알이 화면을 벗어나면 제거
        if v.y <= 100:
            del_bulList.append(i)
    for i in del_bulList:
        del bulList[i]
    """
    ============================================================================================================
    """
    
    
    """
    ============================================================================================================
    4.3 아이템 생성
    makeItem : 몇 킬 마다 아이템 나오게 할지 정하는 변수
    """
    #지금은 해당 shootDown이 되면 화면을 장악함, move도 안됨.
    #그래서 만들면 True False로 구분해야 할 듯
    
    
    
    #더 이상 체력 증가를 할 수 없다면 체력 아이템은 안나옴.
    if bullet_speed <= 7 and lifeNum-hero.kill >= 10:
        itemNum = random.randrange(2,4)
    elif bullet_speed <= 7:
        itemNum = random.randrange(1,4)
    elif lifeNum-hero.kill >= 10:
        itemNum = random.randrange(2,5)
    else:
        itemNum = random.randrange(1,5)
    
    if shootDown != 0 and shootDown % makeItem == 0 and itemCheck == False:
        # healPlus = 체력 증가
        if itemNum == 1:
            healPlus = createObj()
            healPlus.insertImg(r'image\healPlus.png')
            healPlus.setSize(70,70)
            healPlus.x = random.randrange(0, size[0]-healPlus.oX, healPlus.oX)
            healPlus.y = 70
            healPlus.move = 0.7
            healPlusList.append(healPlus)
            shootDown_check = shootDown
            itemCheck=True
        # powerUp = 파워 증가 아이템; 파워는 중요해서 숫자 2개 할당
        elif itemNum in [2,3]:
            powerUp = createObj()
            powerUp.insertImg(r'image\powerUp.png')
            powerUp.setSize(80,80)
            powerUp.x = random.randrange(0, size[0]-powerUp.oX, powerUp.oX)
            powerUp.y = 70
            powerUp.move = 0.7
            powerUpList.append(powerUp)
            shootDown_check = shootDown
            itemCheck=True
        # speedUp = 스피드 증가 아이템 몇 개 안나오도록
        elif itemNum == 4:
            speedUp = createObj()
            speedUp.insertImg(r'image\speedUp.png')
            speedUp.setSize(80,80)
            speedUp.x = random.randrange(0, size[0]-speedUp.oX, speedUp.oX)
            speedUp.y = 70
            speedUp.move = 0.7
            speedUpList.append(speedUp)
            shootDown_check = shootDown
            itemCheck=True
            
    # 다시 아이템 나올 수 있도록 값 바뀌면 변수 변환
    if shootDown != shootDown_check:
        if itemCheck == True:
            itemCheck =False
    
    
    """
    ------------------------------------
    아이템 움직이기 및 화면 벗어나면 삭제
    """
    # 생성된 체력 증가 아이템 움직이기
    del_healPlusList=[]
    for i,v in enumerate(healPlusList):
        v.y += v.move
        # 체력 증가 아이템이 화면을 벗어나면 제거
        if v.y <= -v.oY:
            del_healPlusList.append(i)
    for i in del_healPlusList:
        del healPlusList[i]
        
    # 생성된 파워 증가 아이템 움직이기
    del_powerUpList=[]
    for i,v in enumerate(powerUpList):
        v.y += v.move
        # 파워 증가 아이템이 화면을 벗어나면 제거
        if v.y <= -v.oY:
            del_powerUpList.append(i)
    for i in del_powerUpList:
        del powerUpList[i]
        
    # 생성된 스피드 증가 아이템 움직이기
    del_speedUpList=[]
    for i,v in enumerate(speedUpList):
        v.y += v.move
        # 스피드 증가 아이템이 화면을 벗어나면 제거
        if v.y <= -v.oY:
            del_speedUpList.append(i)
    for i in del_speedUpList:
        del speedUpList[i]
        
    """
    ------------------------------------
    """
    
    """
    ============================================================================================================
    """
    
    
    
    """
    ============================================================================================================
    4.4 랜덤 적기 생성
    enem_num_index의 값은 0~1사이 값입니다.
    1로 설정하면 적군이 하나도 나오지 않고 1에 가까워질수록 적군이 나오는 빈도수가 줄어듭니다.
    """
    whatEnem = [1,2]
    thisEnem = random.choice(whatEnem)
    if random.random() > enem_num_index:
        enemy = createObj()
        enemy.insertImg(f'image/enemy{thisEnem}.png')
        if thisEnem == 1:
            enemy.setSize(70,70)
        elif thisEnem == 2:
            enemy.setSize(50,70)
        #randrange구간을 설정해줘서 좌우 겹치는 적기 없도록 설정
        enemy.x = random.randrange(0, size[0]-enemy.oX, enemy.oX )
        enemy.y = 70
        enemy.move = random.choice(enemy_speed)
        enemList.append(enemy)
    
    #생성된 적기 움직이기
    del_enemList=[]
    for i,v in enumerate(enemList):
        v.y += v.move
        # 적기가 화면을 벗어나면 제거
        if v.y > size[1]+v.oY:
            del_enemList.append(i)
    for i in del_enemList:
        del enemList[i]
        missed+=1             #적기 놓치면 증가
        point-=50             #적기 놓치면 -50
        
    """
    ============================================================================================================
    """
    
    
    
    """
    ============================================================================================================
    4.5 적군과 총알이 충돌하면 일어나는 이벤트를 정의한 구간입니다.
    deathNum의 값을 통해서 적군의 HP를 설정할 수 있습니다.
    """
    
    cd_del_enemList=[]        #피격 적군 삭제 리스트
    cd_del_bulList=[]         #타격 총알 삭제 리스트
    del_hitList=[]            #피격 시 이벤트 삭제 리스트
    for i,enem in enumerate(enemList):
        for j,bul in enumerate(bulList):
            if crashDetec(enem,bul):             #타격 시
                
                
                """
                ----------------------
                타격 시 타격 효과 생성
                """
                hit = createObj()
                hit.insertImg(r'image\hit.png')
                hit.setSize(80,80)
                eX,eY = crashCoor(enem,bul)
                hit.x = eX - round(hit.oX/2) + 20
                hit.y = eY - round(hit.oY/2)
                hit.tic = t.time()
                hitList.append(hit)
                """
                ----------------------
                """
                
                #총알 삭제 리스트에 추가
                enem.kill+=hero.power
#                 cd_del_bulList.append(j)
                del bulList[j]
                
                #격추 판정되면 적군 제거 리스트 추가
                if enem.kill>=deathNum:
                    break_sound.stop()
                    break_sound.play()
                    """
                    ---------------------------
                    격추 판정 시 격추 효과 생성
                    """
                    boom=createObj()
                    boom.insertImg(r'image\boom.png')
                    boom.setSize(90,90)
                    boom.x = eX - round(boom.oX/2) + 20
                    boom.y = eY - round(boom.oY/2)
                    boom.tic = t.time()
                    boomList.append(boom)
                    """
                    ---------------------------
                    """
                    
#                     cd_del_enemList.append(i)
                    del enemList[i]
                    shootDown += 1           #적군 격추 시 증가
                    point += 100             #적군 격추 시 포인트 100 증가
                    enem.kill=0
                    
    # 시간이 지나면 적군 목숨이 강해짐.
    # 현재 change_time = 5
    if sub_time != 0 and sub_time % change_time == 0 and update_deathNum == False:
        deathNum += 1
        if enem_num_index >= 0.91 and enem_num_check % 7 in [0,1,3,4,6]:
            enem_num_index -= 0.02
            enem_num_check+=1
        elif enem_num_check % 8 in [2,5]:
            enem_num_check+=1
            for i,v in enumerate(enemy_speed):
                enemy_speed[i] = v+1
        elif enem_num_check % 8 == 7:
            enemy_speed[-1] += 1
            enem_num_check+=1
        update_deathNum = True
    
    # 다시 적군 HP 증가하도록 변수 변환
    if update_deathNum == True and sub_time % change_time != 0:
        update_deathNum = False
                    
                    
    """
    ---------------------------------
    타격 효과 삭제
    hit_time으로 인해 유지 시간 정함.
    """
    hit_time = 0.1
    for idx, hit in enumerate(hitList):
        hit_toc=t.time()
        if round(hit.tic,1) + hit_time <= round(hit_toc,1):
            del hitList[idx]
    """
    ---------------------------------
    """
    
    """
    ---------------------------------
    격추 효과 삭제
    boom_time으로 인해 유지 시간 정함.
    """
    boom_time = 0.5
    for idx,boom in enumerate(boomList):
        boom_toc=t.time()
        if round(boom.tic,1)+boom_time <= round(boom_toc,1):
            del boomList[idx]
    """
    ---------------------------------
    """

    
    """
    -----------------------------
    피격, 타격으로 인한 객체 삭제
    """    
#     cd_del_enemList=list(set(cd_del_enemList))  #중복 피격 방지
#     cd_del_bulList=list(set(cd_del_bulList))    #중복 타격 방지
    
#     cd_del_enemList.reverse()
#     cd_del_bulList.reverse()

#     #삭제 코드
#     for i in cd_del_enemList:
#         del enemList[i]          #적기 제거
#         shootDown += 1           #적군 격추 시 증가
#         point += 100             #적군 격추 시 포인트 100 증가
        
#     for i in cd_del_bulList:
#         del bulList[i]           #총알 제거

    
        
    """
    -----------------------------
    """
        
    """
    ============================================================================================================
    """
    
    
    """
    ============================================================================================================
    4.6 캐릭터와 적군이 충돌할 때 일어나는 이벤트를 정의한 구간입니다.
    lifeNum 변수를 통해서 본체의 HP를 설정할 수 있습니다.
    """
    
    #invin == True면 무적 False면 무적 아님.
    del_lifeList=[]
    for i,enem in enumerate(enemList):
        if crashDetec(hero,enem) and  invin==False:
            heartBeet_sound.stop()
            heartBeet_sound.play()
            invin_tic = t.time()
            invin = True
            hero.kill += 1
            del lifeList[-1]        #목숨 하나 제거
            
            #무적상태일 때 레드 스크린 효과 적용
            redScreen = createObj()
            redScreen.insertImg(r'image\redScreen.png')
            redScreen.setSize(800,1000)
            hit.x = 0
            hit.y = 0
            redList.append(redScreen)
            
            if hero.kill == lifeNum:
                end=0               #게임 종료
                game_over=1          #게임 오버
    
    invin_toc = t.time()
    if round(invin_tic+invinTime) == round(invin_toc):          #무적 끝나면 invin = False로 만들어서 무적 풀고
        if invin == True:
            del redList[-1]                 #레드 스크린 효과 종료
            invin = False
        
    """
    ============================================================================================================
    """
    
    """
    ============================================================================================================
    4.7 아이템 이벤트 정의
    """   
    
    """
    --------------------
    healPlus 이벤트 정의
    """
    for idx, heal in enumerate(healPlusList):
        if crashDetec(hero,heal):
            item_sound.stop()
            item_sound.play()
            del healPlusList[idx]
            if lifeNum-hero.kill <=9:
                life = createObj()
                life.insertImg(r'image\life.png')
                life.setSize(40,40)
                life.x = 105 + (lifeNum-hero.kill)*50
                life.y = 5
                lifeList.append(life)
                lifeNum+=1
    """
    --------------------
    """
    
    """
    -------------------
    powerUp 이벤트 정의
    """
    for idx, powerUp in enumerate(powerUpList):
        if crashDetec(hero,powerUp):
            item_sound.play()
            del powerUpList[idx]
            hero.power+=1
    """
    -------------------
    """
    
    """
    -------------------
    speedUp 이벤트 정의
    """
    for idx, speedUp in enumerate(speedUpList):
        if crashDetec(hero,speedUp):
            item_sound.play()
            del speedUpList[idx]
            hero.move+=1
            bullet_speed -= 1
    """
    -------------------
    """
    
    
    
    """
    ============================================================================================================
    """
        
    
    """
    Game 5. 그리기
    """
    
    """
    ============================================================================================================
    5.1 이미지 객체 띄우기
    """
    #화면 색 채우기
    screen.blit(backgroundImg,(0,0))
    # 상태창 보더
    screen.blit(border,(0,0))
    
    #캐릭터 띄우기
    hero.show()
    
    #총알 띄우기
    for v in bulList:
        v.show()
        
    #적기 띄우기
    for v in enemList:
        v.show()
    
    # 타격 효과 띄우기
    for v in hitList:
        v.show()
    
    # hero 생명 띄우기
    for v in lifeList:
        v.show()
        
    # 체력 아이템 띄우기
    for v in healPlusList:
        v.show()
    
    # 파워 업 아이템 띄우기
    for v in powerUpList:
        v.show()
    
    # 스피드 업 아이템 띄우기
    for v in speedUpList:
        v.show()
        
    # 격추 효과 띄우기
    for v in boomList:
        v.show()
        
    
        
    """
    ============================================================================================================
    """
    
    
    """
    ============================================================================================================
    5.2 텍스트 표시 절차
    """
    #1. 폰트 설정(폰트 주소, 폰트 크기)
    font = pygame.font.Font(r"font\경기천년제목V_Bold.ttf",20)
    font2 = pygame.font.Font(r"font\경기천년제목V_Bold.ttf",30)
    
    #2. surface 생성(텍스트, 안티앨리어싱, 폰트 색(rgb))
    #shootDown 값과 missed값 표시
    text_shoot = font.render(f"Shoot Down : {shootDown}",True, (255,255,255))
    text_missed = font.render(f"Missed : {missed}",True, (255,255,255))
    text_time = font.render(f"Time : {sub_time}",True,(255,255,255))
    text_life = font2.render("Life : ",True,(255,255,255))
    text_power = font2.render(f"Power : {hero.power}",True,(255,255,255))
    
    
    
    #3. surface 표시(표시할 이미지, 위치)
    screen.blit(text_shoot, (590,15))
    screen.blit(text_missed, (590,35))
    screen.blit(text_time, (590,55))
    screen.blit(text_life, (27,15))
    screen.blit(text_power, (27,55))
    
    """
    무적효과를 제일 마지막에 띄우도록 함.
    """
    # 무적 효과 띄우기
    for v in redList:
        v.show()
    
    """
    ============================================================================================================
    """

    """
    Game 6. 업데이트
    """
    #screen 상태 업데이트
    pygame.display.flip()
    if game_over == 1:
        end_game(point,shootDown,missed,sub_time,game_over)

        
        
"""
=====================================================================================================================
=====================================================================================================================
=====================================================================================================================
=====================================================================================================================
"""        

# 게임 종료
pygame.quit()

In [ ]:
pip install pygame